# Import Dependincies 

In [1]:
import pandas as pd
import spacy
import nltk
from collections.abc import Mapping
from experta import *
from spacy.matcher import PhraseMatcher
from experta import KnowledgeEngine, Fact, Rule, Field
nlp = spacy.load("en_core_web_sm")

In [14]:
df = pd.read_csv('Medical Diagnosis Expert System.csv')


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   disease      4920 non-null   object
 1   symptoms     4920 non-null   object
 2   precautions  4560 non-null   object
dtypes: object(3)
memory usage: 115.4+ KB


In [3]:
symptom_phrases = set()
for symptoms in df['symptoms']:
    for symptom in symptoms.split(','):
        symptom_phrases.add(symptom.replace('_', ' ').strip())

matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
patterns = [nlp.make_doc(symptom) for symptom in symptom_phrases]
matcher.add("SYMPTOM", patterns)

def preprocess_input(user_input):
    """
    Extracts known symptom phrases from user input using PhraseMatcher.
    Returns a list of matched symptoms from the dataset.
    """
    formatted_input = user_input.lower().replace('_', ' ')
    doc = nlp(formatted_input)
    matches = matcher(doc)

    extracted_symptoms = set()
    for match_id, start, end in matches:
        span = doc[start:end]
        extracted_symptoms.add(span.text.strip())

    return list(extracted_symptoms)

In [4]:
example_sentence = "I have a severe headache and feel very fatigued with a runny nose."
processed_example_no_verbs = preprocess_input(example_sentence)
print("Example Sentence:")
print(example_sentence)
print("Processed Symptoms ")
print(processed_example_no_verbs)

Example Sentence:
I have a severe headache and feel very fatigued with a runny nose.
Processed Symptoms 
['runny nose', 'headache']


In [5]:
print(type (df['disease'][0]))
print(type (df['symptoms'][0]))
print(type (df['precautions'][0]))

<class 'str'>
<class 'str'>
<class 'str'>


In [6]:
disease_data = {}

for _, row in df.iterrows():
    disease = row['disease']
    if isinstance(row['symptoms'], list):
        raw_symptoms = [s.strip() for s in row['symptoms']]
    else:
        raw_symptoms = [s.strip() for s in str(row['symptoms']).split(',')]
    processed_symptoms = set()
    for s in raw_symptoms:
        processed_symptoms.update(preprocess_input(s))

    precautions = [p.strip() for p in str(row['precautions']).split(',') if p]

    if disease not in disease_data:
        disease_data[disease] = {'symptoms': set(), 'precautions': set()}
    disease_data[disease]['symptoms'].update(processed_symptoms)
    disease_data[disease]['precautions'].update(precautions)

for disease in disease_data:
    disease_data[disease]['symptoms'] = sorted(disease_data[disease]['symptoms'])
    disease_data[disease]['precautions'] = sorted(disease_data[disease]['precautions'])


In [7]:
print(disease_data['Fungal infection']['symptoms'])
print(disease_data['Fungal infection']['precautions'])

['dischromic  patches', 'itching', 'nodal skin eruptions', 'skin rash']
['bath twice', 'keep infected area dry', 'use clean cloths', 'use detol or neem in bathing water']


In [8]:
print("Number of diseases:", len(disease_data))

Number of diseases: 38


In [9]:

class Symptom(Fact):
    name = Field(str, mandatory=True)

class Diagnosis(Fact):
    disease = Field(str, mandatory=True)

class DiseaseDiagnosisEngine(KnowledgeEngine):
    def __init__(self, disease_data, input_symptoms):
        super().__init__()
        self.disease_data = disease_data
        self.input_symptoms = set(input_symptoms)
        self.possible_diseases = []
        self.matching_diseases = set()

    @Rule(Symptom(name=MATCH.symptom_name))
    def check_symptom(self, symptom_name):
        for disease, info in self.disease_data.items():
            if symptom_name in info['symptoms']:
                if not hasattr(self, f"matched_symptoms_{disease}"):
                    setattr(self, f"matched_symptoms_{disease}", {symptom_name})
                else:
                    getattr(self, f"matched_symptoms_{disease}").add(symptom_name)

    def get_diagnosis(self):
        self.matching_diseases = set()
        self.reset()
        for symptom in self.input_symptoms:
            self.declare(Symptom(name=symptom))
        self.run()

        for disease in self.disease_data:
            matched_symptoms = getattr(self, f"matched_symptoms_{disease}", set())
            if self.input_symptoms.issubset(matched_symptoms):
                self.matching_diseases.add(disease)
            if hasattr(self, f"matched_symptoms_{disease}"):
                delattr(self, f"matched_symptoms_{disease}")

        print("Diseases with all provided symptoms:", list(self.matching_diseases))
        return list(self.matching_diseases)

In [10]:
def get_precautions(disease_name):
    return disease_data.get(disease_name, {}).get('precautions', [])



In [11]:

def main():
    print("\nMedical Diagnosis Expert System")
    print("--------------------------------")
    print("Describe your symptoms (e.g., 'I have fever and headache'):")

    while True:
        user_input = input("\n> ").strip()

        if user_input.lower() in ['exit', 'quit']:
            print("Goodbye!")
            break

        if not user_input:
            print("Please enter your symptoms.")
            continue

        symptoms = preprocess_input(user_input)
  
        if not symptoms:
            print("No relevant symptoms detected. Please try again.")
            continue

        print(f"\n[Extracted Symptoms]: {', '.join(symptoms)}")

        print("\n[Diagnosis Results]")
        print("-------------------")


        engine = DiseaseDiagnosisEngine(disease_data, symptoms)
        engine.reset()
        diseases = engine.get_diagnosis()
        
        # print(f"Possible Diseases: {', '.join(diseases)}")

        if diseases:
            precautions = get_precautions(diseases[0])
            print(f"Diagnosis: {diseases[0]}")
            print("\nRecommended Precautions:")
            for i, precaution in enumerate(precautions, 1):
                print(f"{i}. {precaution}")
        else:
            print("No matching diseases found.")
            print("Please consult a healthcare professional.")
        
        print("\nEnter more symptoms or type 'exit' to quit.")


if __name__ == "__main__":
    main()


Medical Diagnosis Expert System
--------------------------------
Describe your symptoms (e.g., 'I have fever and headache'):

[Extracted Symptoms]: malaise, headache, mild fever, lethargy, high fever, swelled lymph nodes, red spots over body, itching, loss of appetite, skin rash

[Diagnosis Results]
-------------------
Diseases with all provided symptoms: ['Chicken pox']
Diagnosis: Chicken pox

Recommended Precautions:
1. avoid public places
2. consume neem leaves
3. take vaccine
4. use neem in bathing

Enter more symptoms or type 'exit' to quit.
Please enter your symptoms.

[Extracted Symptoms]: yellowing of eyes, yellowish skin, acute liver failure, high fever, vomiting, loss of appetite, coma, stomach bleeding, abdominal pain, nausea, dark urine

[Diagnosis Results]
-------------------
Diseases with all provided symptoms: ['Hepatitis E']
Diagnosis: Hepatitis E

Recommended Precautions:
1. consult doctor
2. medication
3. rest
4. stop alcohol consumption

Enter more symptoms or type 